In [ ]:
import json
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-8B")


In [ ]:
ANIMATION_TIME_SCALE = 1

files = ['mbpp_samples.json', 'humaneval_samples.json', 'gsm8k_cot_samples.json', 'minerva_math_samples.json']

all_streaming_data_neo_entry = []
for id, file in enumerate(files):
    with open(f'./{file}', 'r') as f:
        data = json.load(f)

    tidar_steps =data['models']['tidar_8b']['steps']

    # create a new json file, that contains a list of decoded tidar steps
    already_decoded_tokens = []
    decoding_progress = []
    cum_tokens = 0
    for i in range(len(tidar_steps)):
        already_accepted_tokens = tokenizer.decode(already_decoded_tokens, skip_special_tokens=True)
        already_decoded_tokens.extend(tidar_steps[i][0][:tidar_steps[i][1]])
        draft_tokens = tokenizer.decode(tidar_steps[i][0], skip_special_tokens=True)
        
        decoding_progress.append((already_accepted_tokens, draft_tokens, cum_tokens))
        cum_tokens += tidar_steps[i][1]

    speculation_data = {}

    speculation_data['name'] = 'TiDAR-8B'
    speculation_data['link'] = 'https://arxiv.org/pdf/2511.08923'
    speculation_data['speculative_progress'] = decoding_progress
    speculation_data['response'] = decoding_progress[-1][0] + decoding_progress[-1][1]
    speculation_data['actual_stream_time'] = data['models']['tidar_8b']['decode_latency'] / 512 
    speculation_data['animation_time'] = speculation_data['actual_stream_time'] * ANIMATION_TIME_SCALE
    speculation_data['total_tokens'] = cum_tokens

    streaming_data_neo_entry = {}
    streaming_data_neo_entry['id'] = id

    if file == 'gsm8k_cot_samples.json':
        streaming_data_neo_entry['question'] = "[GSM-8K] Q:" + data['prompt'].split("Q:")[-1]
        streaming_data_neo_entry['actual_prompt'] = "[GSM-8K] ... (8-shot prompt) ...\nQ:" + data['prompt'].split("Q:")[-1]
    elif file == 'mbpp_samples.json':
        streaming_data_neo_entry['question'] = "[MBPP] You are an expert Python programmer" + data['prompt'].split("You are an expert Python programmer")[-1]
        streaming_data_neo_entry['actual_prompt'] = "[MBPP]... (3-shot prompt) ...\nYou are an expert Python programmer" + data['prompt'].split("You are an expert Python programmer")[-1]
    elif file == 'minerva_math_samples.json':
        streaming_data_neo_entry['question'] = "[Minerva Math] Problem:" + data['prompt'].split("Problem:")[-1]
        streaming_data_neo_entry['actual_prompt'] = "[Minerva Math] ... (4-shot prompt) ...\nProblem:" + data['prompt'].split("Problem:")[-1]
    else:
        streaming_data_neo_entry['question'] = "[HumanEval] " + data['prompt']
        streaming_data_neo_entry['actual_prompt'] = "[HumanEval]" + data['prompt']
    
    streaming_data_neo_entry['model_a'] = speculation_data
    streaming_data_neo_entry['model_b'] = {
        'name': 'Qwen3-8B-Base',
        'link': 'https://huggingface.co/Qwen/Qwen3-8B-Base',
        'response': data['models']['qwen3_8b']['output'],
        'total_tokens': speculation_data['total_tokens'],
        'actual_stream_time': data['models']['qwen3_8b']['decode_latency'] / 512 ,
        'animation_time': data['models']['qwen3_8b']['decode_latency'] / 512 * ANIMATION_TIME_SCALE,
    }
    all_streaming_data_neo_entry.append(streaming_data_neo_entry)

with open('../streaming_data_neo.json', 'w') as f:
    json.dump(all_streaming_data_neo_entry, f)
